# Lab 1


## Part 1: Bilingual dictionary induction and unsupervised embedding-based MT (30%)
*Note: this homework is based on materials from yandexdataschool [NLP course](https://github.com/yandexdataschool/nlp_course/). Feel free to check this awesome course if you wish to dig deeper.*

*Refined by [Nikolay Karpachev](https://www.linkedin.com/in/nikolay-karpachev-b0146a104/)*

**In this homework** **<font color='red'>YOU</font>** will make machine translation system without using parallel corpora, alignment, attention, 100500 depth super-cool recurrent neural network and all that kind superstuff.

But even without parallel corpora this system can be good enough (hopefully), in particular for similar languages, e.g. Ukrainian and Russian. 

### Frament of the Swadesh list for some slavic languages

The Swadesh list is a lexicostatistical stuff. It's named after American linguist Morris Swadesh and contains basic lexis. This list are used to define subgroupings of languages, its relatedness.

So we can see some kind of word invariance for different Slavic languages.


| Russian         | Belorussian              | Ukrainian               | Polish             | Czech                         | Bulgarian            |
|-----------------|--------------------------|-------------------------|--------------------|-------------------------------|-----------------------|
| женщина         | жанчына, кабета, баба    | жінка                   | kobieta            | žena                          | жена                  |
| мужчина         | мужчына                  | чоловік, мужчина        | mężczyzna          | muž                           | мъж                   |
| человек         | чалавек                  | людина, чоловік         | człowiek           | člověk                        | човек                 |
| ребёнок, дитя   | дзіця, дзіцёнак, немаўля | дитина, дитя            | dziecko            | dítě                          | дете                  |
| жена            | жонка                    | дружина, жінка          | żona               | žena, manželka, choť          | съпруга, жена         |
| муж             | муж, гаспадар            | чоловiк, муж            | mąż                | muž, manžel, choť             | съпруг, мъж           |
| мать, мама      | маці, матка              | мати, матір, неня, мама | matka              | matka, máma, 'стар.' mateř    | майка                 |
| отец, тятя      | бацька, тата             | батько, тато, татусь    | ojciec             | otec                          | баща, татко           |
| много           | шмат, багата             | багато                  | wiele              | mnoho, hodně                  | много                 |
| несколько       | некалькі, колькі         | декілька, кілька        | kilka              | několik, pár, trocha          | няколко               |
| другой, иной    | іншы                     | інший                   | inny               | druhý, jiný                   | друг                  |
| зверь, животное | жывёла, звер, істота     | тварина, звір           | zwierzę            | zvíře                         | животно               |
| рыба            | рыба                     | риба                    | ryba               | ryba                          | риба                  |
| птица           | птушка                   | птах, птиця             | ptak               | pták                          | птица                 |
| собака, пёс     | сабака                   | собака, пес             | pies               | pes                           | куче, пес             |
| вошь            | вош                      | воша                    | wesz               | veš                           | въшка                 |
| змея, гад       | змяя                     | змія, гад               | wąż                | had                           | змия                  |
| червь, червяк   | чарвяк                   | хробак, черв'як         | robak              | červ                          | червей                |
| дерево          | дрэва                    | дерево                  | drzewo             | strom, dřevo                  | дърво                 |
| лес             | лес                      | ліс                     | las                | les                           | гора, лес             |
| палка           | кій, палка               | палиця                  | patyk, pręt, pałka | hůl, klacek, prut, kůl, pálka | палка, пръчка, бастун |

But the context distribution of these languages demonstrates even more invariance. And we can use this fact for our for our purposes.

## Data

In [1]:
import gensim
import numpy as np
from gensim.models import KeyedVectors

In this notebook we're going to use pretrained word vectors - FastText (original paper - https://arxiv.org/abs/1607.04606).

You can download them from the official [website](https://fasttext.cc/docs/en/crawl-vectors.html). We're going to need embeddings for Russian and Ukrainian languages. Please use word2vec-compatible format (.text).

In [270]:
uk_emb = KeyedVectors.load_word2vec_format("cc.uk.300.vec")

In [271]:
ru_emb = KeyedVectors.load_word2vec_format("cc.ru.300.vec")

In [272]:
ru_emb.most_similar([ru_emb["август"]], topn=10)

[('август', 1.0),
 ('июль', 0.9383153319358826),
 ('сентябрь', 0.9240028858184814),
 ('июнь', 0.9222576022148132),
 ('октябрь', 0.9095539450645447),
 ('ноябрь', 0.8930035829544067),
 ('апрель', 0.8729087114334106),
 ('декабрь', 0.8652557730674744),
 ('март', 0.8545796871185303),
 ('февраль', 0.8401416540145874)]

In [273]:
uk_emb.most_similar([uk_emb["серпень"]])

[('серпень', 1.0),
 ('липень', 0.9096439480781555),
 ('вересень', 0.9016969203948975),
 ('червень', 0.8992519974708557),
 ('жовтень', 0.8810408115386963),
 ('листопад', 0.8787633776664734),
 ('квітень', 0.8592804670333862),
 ('грудень', 0.8586863279342651),
 ('травень', 0.8408110737800598),
 ('лютий', 0.8256431221961975)]

In [274]:
ru_emb.most_similar([uk_emb["серпень"]])

[('Stepashka.com', 0.2757962942123413),
 ('ЖИЗНИВадим', 0.25203439593315125),
 ('2Дмитрий', 0.25048112869262695),
 ('2012Дмитрий', 0.24829229712486267),
 ('Ведущий-Алексей', 0.2443869709968567),
 ('Недопустимость', 0.24435287714004517),
 ('2Михаил', 0.23981398344039917),
 ('лексей', 0.23740758001804352),
 ('комплексн', 0.23695147037506104),
 ('персональ', 0.2368222177028656)]

Load small dictionaries for correspoinding words pairs as trainset and testset.

In [275]:
def load_word_pairs(filename):
    uk_ru_pairs = []
    uk_vectors = []
    ru_vectors = []
    with open(filename, "r", encoding="utf-8") as inpf:
        for line in inpf:
            uk, ru = line.rstrip().split("\t")
            if uk not in uk_emb or ru not in ru_emb:
                continue
            uk_ru_pairs.append((uk, ru))
            uk_vectors.append(uk_emb[uk])
            ru_vectors.append(ru_emb[ru])
    return uk_ru_pairs, np.array(uk_vectors), np.array(ru_vectors)

In [276]:
#!wget -O ukr_rus.train.txt http://tiny.cc/jfgecz

In [277]:
#!wget -O ukr_rus.test.txt http://tiny.cc/6zoeez

In [278]:
uk_ru_train, X_train, Y_train = load_word_pairs("ukr_rus.train.txt")

In [279]:
uk_ru_test, X_test, Y_test = load_word_pairs("ukr_rus.test.txt")

## Embedding space mapping (0.3 pts)

Let $x_i \in \mathrm{R}^d$ be the distributed representation of word $i$ in the source language, and $y_i \in \mathrm{R}^d$ is the vector representation of its translation. Our purpose is to learn such linear transform $W$ that minimizes euclidian distance between $Wx_i$ and $y_i$ for some subset of word embeddings. Thus we can formulate so-called Procrustes problem:

$$W^*= \arg\min_W \sum_{i=1}^n||Wx_i - y_i||_2$$
or
$$W^*= \arg\min_W ||WX - Y||_F$$

where $||*||_F$ - Frobenius norm.

$W^*= \arg\min_W \sum_{i=1}^n||Wx_i - y_i||_2$ looks like simple multiple linear regression (without intercept fit). So let's code.

In [280]:
from sklearn.linear_model import LinearRegression

# YOUR CODE HERE
mapping = LinearRegression()

# Train the model using the training sets
mapping.fit(X_train, Y_train)

# Make predictions using the testing set
Y_pred = mapping.predict(X_test)
W = mapping.coef_

Let's take a look at neigbours of the vector of word _"серпень"_ (_"август"_ in Russian) after linear transform.

In [281]:
august = mapping.predict(uk_emb["серпень"].reshape(1, -1))
ru_emb.most_similar(august)

[('апрель', 0.8531404137611389),
 ('июнь', 0.8402308225631714),
 ('март', 0.8385775089263916),
 ('сентябрь', 0.8331868648529053),
 ('февраль', 0.8311494588851929),
 ('октябрь', 0.8278172016143799),
 ('ноябрь', 0.8244151473045349),
 ('июль', 0.822899580001831),
 ('август', 0.8112362623214722),
 ('январь', 0.8022860288619995)]

In [282]:
len([("серпень", "август")]), len(august)

(1, 1)

We can see that neighbourhood of this embedding consists of different months, but right variant is on the ninth place.

As quality measure we will use precision top-1, top-5 and top-10 (for each transformed Ukrainian embedding we count how many right target pairs are found in top N nearest neighbours in Russian embedding space).

In [327]:
def precision(pairs, mapped_vectors, topn=1):
    """
    :args:
        pairs = list of right word pairs [(uk_word_0, ru_word_0), ...]
        mapped_vectors = list of embeddings after mapping from source embedding space to destination embedding space
        topn = the number of nearest neighbours in destination embedding space to choose from
    :returns:
        precision_val, float number, total number of words for those we can find right translation at top K.
    """
    assert len(pairs) == len(mapped_vectors)
    num_matches = 0
    for i, (_, ru) in enumerate(pairs):
        # YOUR CODE HERE
        #word = mapping.predict(uk_emb[pairs[i][0]].reshape(1, -1))
        word = mapped_vectors[i]
        similar = ru_emb.most_similar([word], topn=topn)
        #print(similar)
        for el in similar:
            num_matches += (el[0] == ru)
    precision_val = num_matches / len(pairs)
    return precision_val

In [328]:
assert precision([("серпень", "август")], august, topn=5) == 0.0
assert precision([("серпень", "август")], august, topn=9) == 1.0
assert precision([("серпень", "август")], august, topn=10) == 1.0

In [331]:
assert precision(uk_ru_test, X_test) == 0.0
assert precision(uk_ru_test, Y_test) == 1.0

In [332]:
precision_top1 = precision(uk_ru_test, mapping.predict(X_test), 1)
precision_top5 = precision(uk_ru_test, mapping.predict(X_test), 5)

In [333]:
print(precision_top1)
print(precision_top5)

0.6259541984732825
0.7913486005089059


## Making it better (orthogonal Procrustean problem) (0.3 pts)

It can be shown (see original paper) that a self-consistent linear mapping between semantic spaces should be orthogonal. 
We can restrict transform $W$ to be orthogonal. Then we will solve next problem:

$$W^*= \arg\min_W ||WX - Y||_F \text{, where: } W^TW = I$$

$$I \text{- identity matrix}$$

Instead of making yet another regression problem we can find optimal orthogonal transformation using singular value decomposition. It turns out that optimal transformation $W^*$ can be expressed via SVD components:
$$X^TY=U\Sigma V^T\text{, singular value decompostion}$$
$$W^*=UV^T$$

In [334]:
import numpy as np

In [399]:
def learn_transform(X_train, Y_train):
    """ 
    :returns: W* : float matrix[emb_dim x emb_dim] as defined in formulae above
    """
    from sklearn.decomposition import TruncatedSVD
    svd = TruncatedSVD(299)
    matr = np.transpose(X_train) @ Y_train
    U = svd.fit_transform(matr) / svd.singular_values_
    VT = svd.components_
    mapping = np.matmul(U, VT)  # output of TruncatedSVD
    # YOUR CODE GOES HERE
    # compute orthogonal embedding space mapping
    # mapping = ...

    return mapping

In [400]:
W = learn_transform(X_train, Y_train)

In [401]:
ru_emb.most_similar([np.matmul(uk_emb["серпень"], W)], topn=10)

[('апрель', 0.8245381116867065),
 ('июнь', 0.8056143522262573),
 ('сентябрь', 0.8055691719055176),
 ('март', 0.8033100366592407),
 ('октябрь', 0.7987180948257446),
 ('июль', 0.7946286201477051),
 ('ноябрь', 0.7939659357070923),
 ('август', 0.7937638163566589),
 ('февраль', 0.7923961281776428),
 ('декабрь', 0.7715296745300293)]

In [402]:
print(precision(uk_ru_test, np.matmul(X_test, W)))
print(precision(uk_ru_test, np.matmul(X_test, W), 5))

0.6463104325699746
0.7989821882951654


## Unsupervised embedding-based MT (0.4 pts)

Now, let's build our word embeddings-based translator!

Firstly, download OPUS Tatoeba corpus.

In [403]:
#!wget https://object.pouta.csc.fi/OPUS-Tatoeba/v20190709/mono/uk.txt.gz

In [404]:
#!gzip -d ./uk.txt.gz

In [605]:
with open('./uk.txt', 'r', encoding="utf8") as f:
    uk_corpus = f.readlines()

In [606]:
# To save your time and CPU, feel free to use first 1000 sentences of the corpus
uk_corpus = uk_corpus[:1000]

In [607]:
uk_corpus[:5]

['Я вже закінчу коледж, коли ви вернетеся з Америки.\n',
 'Він наказав мені негайно вийти з кімнати.\n',
 'Як би ти не намагався, ти не вивчиш англійську за два-три місяці.\n',
 'Поки я не подзвонив, він не прийшов.\n',
 'У всесвіті багато галактик.\n']

In [609]:
from nltk.tokenize import WordPunctTokenizer

tokenizer = WordPunctTokenizer()
uk_corpus = [tokenizer.tokenize(x.lower()) for x in uk_corpus]

uk_corpus[:5]

[['я',
  'вже',
  'закінчу',
  'коледж',
  ',',
  'коли',
  'ви',
  'вернетеся',
  'з',
  'америки',
  '.'],
 ['він', 'наказав', 'мені', 'негайно', 'вийти', 'з', 'кімнати', '.'],
 ['як',
  'би',
  'ти',
  'не',
  'намагався',
  ',',
  'ти',
  'не',
  'вивчиш',
  'англійську',
  'за',
  'два',
  '-',
  'три',
  'місяці',
  '.'],
 ['поки', 'я', 'не', 'подзвонив', ',', 'він', 'не', 'прийшов', '.'],
 ['у', 'всесвіті', 'багато', 'галактик', '.']]

In [628]:
def translate(sentence):
    """
    :args:
        sentence - sentence in Ukrainian (str)
    :returns:
        translation - sentence in Russian (str)

    * find ukrainian embedding for each word in sentence
    * transform ukrainian embedding vector
    * find nearest russian word and replace
    """
    #sentence = (sentence).split()
    translated = []
    # YOUR CODE GOES HERE
    if type(sentence) == str:
        sentence = sentence.split()
    for word in sentence:
        if word == ' ':
            continue
        elif word not in uk_emb:
            translated.append('UNK')
        else:
            translated.append(ru_emb.most_similar([np.matmul(uk_emb[word], W)], topn=1)[0][0])
    return ' '.join(translated)

In [629]:
assert translate(".") == "."
assert translate("1 , 3") == "1 , 3"
assert translate("кіт зловив мишу") == "кот поймал мышку"

Now you can play with your model and try to get as accurate translations as possible. **Note**: one big issue is out-of-vocabulary words. Try to think of various ways of handling it (you can start with translating each of them to a special **UNK** token and then move to more sophisticated approaches). Good luck!

In [630]:
for sent in uk_corpus[::10]:
    print(translate(sent))

мной уже закончу колледж , когда мы прибежишь со америки .
город бомбили враждебные самолеты .
возможно , мной антисоциальный , конечно это не означает , что мной не общаюсь со людьми .
этого утра выпала роса .
беда не приходит одна .
посмотри по тот дым .
мной заказал два гамбургера .
мной не хотел никого обидеть .
гора покрыта снегом .
по фотографии во девушки корона не со золота , а со цветов .
во меня То мечта .
мной приехал во UNK со китая .
по север находится UNK ; по юге — англия ; по востоке — англо-саксонский ; и ещe дальше по востоке — северная шотландия .
его родная страна — германия .
Уотертаун — столица ирландии .
он ждал по него к десятой часа .
ты можешь взять ту книгу даром .
этот роман сочинил известный американский писатель .
забронировать , будте ласковые , комнату возле международного аэропорта во UNK .
он знает , что ты его влюбится ?
мной знаю , что ты богатый .
те , кто всё забывают , счастливые .
во этой реке опасно плавать .
пришел , увидел , победил .
мной хож

Great! 
See second notebook for the Neural Machine Translation assignment.

In [640]:
def translate_smart(sentence):
    translated = []
    # YOUR CODE GOES HERE
    if type(sentence) == str:
        sentence = sentence.split()
    for word in sentence:
        if word == ' ':
            continue
        else:
            while word not in uk_emb:
                word = word[:-1]
        translated.append(ru_emb.most_similar([np.matmul(uk_emb[word], W)], topn=1)[0][0])
    return ' '.join(translated)

In [641]:
for sent in uk_corpus[::10]:
    print(translate_smart(sent))

мной уже закончу колледж , когда мы прибежишь со америки .
город бомбили враждебные самолеты .
возможно , мной антисоциальный , конечно это не означает , что мной не общаюсь со людьми .
этого утра выпала роса .
беда не приходит одна .
посмотри по тот дым .
мной заказал два гамбургера .
мной не хотел никого обидеть .
гора покрыта снегом .
по фотографии во девушки корона не со золота , а со цветов .
во меня То мечта .
мной приехал во нихон со китая .
по север находится Uí ; по юге — англия ; по востоке — англо-саксонский ; и ещe дальше по востоке — северная шотландия .
его родная страна — германия .
Уотертаун — столица ирландии .
он ждал по него к десятой часа .
ты можешь взять ту книгу даром .
этот роман сочинил известный американский писатель .
забронировать , будте ласковые , комнату возле международного аэропорта во зоро .
он знает , что ты его влюбится ?
мной знаю , что ты богатый .
те , кто всё забывают , счастливые .
во этой реке опасно плавать .
пришел , увидел , победил .
мной х